In [36]:
import gym
import numpy as np

#create a single game instance
env = gym.make("Taxi-v2")
#start new game
start = env.reset()
# display the game state
env.render()
n_states = env.observation_space.n
n_actions = env.action_space.n

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [44]:
print("Number of states =", n_states)
print("Number of actions =", n_actions)
print("initial observation code:", start)
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n =', env.observation_space.n)
print("actions:", env.action_space, 'n =', env.action_space.n)

Number of states = 500
Number of actions = 6
initial observation code: 133
printing observation:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
observations: Discrete(500) n = 500
actions: Discrete(6) n = 6


In [39]:
print("taking action 2 (East)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (East)
new observation code: 173
reward: -1
is game over?: False
printing new state:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)


In [40]:
action_to_i = {
    'South':0,
    'North':1,
    'East':2,
    'West':3,
    "Pickup":4,
    "Dropoff":5
}

In [42]:
s,r,done,_=env.step(action_to_i['South'])
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)


In [43]:
done

False

In [45]:
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.randint(0, n_actions, n_states)

In [46]:
np.random.seed(42)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be n_states'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

Action frequencies over 10^4 samples: [ 0.1664472  0.166829   0.1666948  0.1666484  0.1666524  0.1667282]
Seems fine!


In [47]:
def sample_reward(env, policy, t_max=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    for _ in range(t_max):
        action = policy[s]
        s, r, done, _ =  env.step(action)
        total_reward += r
        if done: 
            break
            
    return total_reward

In [49]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'
print("Looks good!")

generating 10^3 sessions...
Looks good!


In [50]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for _ in range(n_times)]
    return float(np.mean(rewards))

In [52]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(10000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)


  0%|          | 1/10000 [00:00<37:43,  4.42it/s]

New best score: -521.02



  0%|          | 2/10000 [00:00<37:39,  4.42it/s]Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  0%|          | 23/10000 [00:04<33:11,  5.01it/s]

New best score: -511.57


  0%|          | 38/10000 [00:07<34:07,  4.87it/s]

...

  8%|▊         | 784/10000 [02:37<30:54,  4.97it/s]

New best score: -413.11


 12%|█▏        | 1178/10000 [03:57<29:36,  4.97it/s]

New best score: -386.2


100%|██████████| 10000/10000 [35:27<00:00,  4.70it/s]


Заметим, что простой перебор работает просто отвратительно:( Попытаемся генерировать генетические алгоритмы

In [53]:
# мутация
# бежим по состояниям и рандомно выбираем 
# между стратегиями с вероятностью p
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    return np.array(
        [np.random.choice([policy1[i], policy2[i]], p=[p, 1 - p]) for i in range(n_states)]
        )

In [54]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(policy, get_random_policy(),p=1-p)
    

In [55]:

n_epochs = 100 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick


In [56]:
print("initializing...")
pool = [get_random_policy() for _ in range(pool_size)]
pool_scores = [evaluate(p) for p in pool]

initializing...


In [57]:
#main loop
from random import choice
for epoch in tqdm(range(n_epochs)):
    print("Epoch %s:"%epoch)
    
    crossovered = [crossover(choice(pool), choice(pool)) for _ in range(n_crossovers)]
    mutated = [mutation(choice(pool))]
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    pool = pool + crossovered + mutated
    pool_scores = [evaluate(p) for p in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])
    #print_policy(pool[-1])

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0:


  1%|          | 1/100 [00:33<54:36, 33.10s/it]

best score: -457.66
Epoch 1:


  2%|▏         | 2/100 [01:05<53:41, 32.87s/it]

...

best score: -100.0
Epoch 98:


 99%|█████████▉| 99/100 [51:10<00:31, 31.01s/it]

best score: -100.0
Epoch 99:


100%|██████████| 100/100 [51:43<00:00, 31.03s/it]

best score: -100.0
